In [1]:
from openmm import app
import openmm as mm
from simtk import unit

In [2]:
pdb = app.PDBFile('butane.pdb')
forcefield = app.ForceField('butane.gaff2.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
# 0.002ps per step
integrator.setConstraintTolerance(1e-5)
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)


In [3]:
print('Minimizing...')
st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")
simulation.minimizeEnergy(maxIterations=100)
st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")


Minimizing...
Potential energy before minimization is 5.797528692127868 kJ/mol
Potential energy after minimization is 5.295784483017486 kJ/mol


In [4]:
from sys import stdout
print('Equilibrating...')
simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(298.0*unit.kelvin)
simulation.step(5000)



Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
250,34.683412853190006,251.05849335802176
500,56.51711419767437,284.00744001473805
750,39.81344164351313,334.72800846263124
1000,48.54532385047364,317.3154709019923
1250,42.09300686448905,454.195759060256
1500,56.53285529742464,275.4578001337142
1750,51.534945446091584,221.2792438728852
2000,51.7901250739522,197.6062240330728
2250,43.33955255104787,402.3314413344983
2500,40.393570285831046,324.8847715999642
2750,31.602906734334063,318.9947908280615
3000,42.77928636592206,394.0756136123654
3250,34.60178298542982,347.85231664884367
3500,49.033565429360074,224.21444387637558
3750,66.21029276523738,319.0453826294694
4000,66.4122482502743,431.91488719338986
4250,39.770574710722485,293.61149777475305
4500,47.09694796090507,253.061539262538
4750,29.678260158332762,364.00114663453127
5000,54.33681946994653,368.61499026532135


In [5]:
import time as time
print('Running Production...')
# Begin timer
tinit=time.time()
# Clear simulation reporters
simulation.reporters.clear()
# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 500000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('butane_sim.dcd', 100))
# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(20000000)
# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
500000,999.9999999901769,29.650005806586343,190.73145210806302,0
1000000,1999.9999999665301,39.090357199445535,231.07796037066396,1.92e+04
1500000,2999.9999999428833,49.898763952655926,312.4203036796892,1.91e+04
2000000,3999.9999999192364,18.81180432008864,283.5359093966083,1.91e+04
2500000,5000.000000101135,64.88554519952912,333.22150347235714,1.92e+04
3000000,6000.000000304862,19.51759172072603,321.11999612626096,1.92e+04
3500000,7000.0000005085885,54.92712989277512,484.6385584817607,1.92e+04
4000000,8000.000000712315,49.00419303885427,417.4327668963519,1.92e+04
4500000,9000.000000916041,50.731237231271884,495.77638225554875,1.92e+04
5000000,10000.000001119768,46.34284814629816,384.16812470229786,1.92e+04
5500000,11000.000001323495,56.31095122238718,259.43443800202584,1.92e+04
6000000,12000.000001527222,72.31658688630567,380.35560582837707,1.92e+04
6500000,13000.000001730948,47.3